## MicroPython ESP32 Experimentation

### Establishing connection to target board
First, make sure you've got the right serial port. On unix-based systems, you can run `ls /dev/tty.*` to see your available serial devices. Replace as necessary below.

This will allow Jupyter (your host computer) to run commands and send/receive information to/from your target board in real time using the MicroPython REPL.

In [113]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
# %serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [114]:
print(1)

1


In [109]:
f = open(".env", "r")
print(f.read())

#.env
WIFI_SSID=TP-Link_AP_4C04
WIFI_PASSWORD=63525465



In [37]:
%ls lib

Listing directory 'lib'.
     8196    lib/.DS_Store
       55    lib/.env
       27    lib/__init__.mpy
        0    lib/__init__.py
             lib/__pycache__/
             lib/aws/
     1554    lib/computation.mpy
     4006    lib/computation.py
      558    lib/config.mpy
      446    lib/config.py
     2041    lib/core.mpy
     5217    lib/core.py
      796    lib/decoding.mpy
     2323    lib/decoding.py
      855    lib/diagnostics.py
     3503    lib/logging.py
     8703    lib/mqtt_as.mpy
      790    lib/networking.mpy
     1254    lib/networking.py
     2654    lib/peripherals.mpy
     5697    lib/peripherals.py
    27364    lib/requests.py
     7664    lib/runner.py
     1016    lib/scheduling.mpy
     1911    lib/scheduling.py
      780    lib/signal.mpy
     2067    lib/signal.py
     2601    lib/umqtt.mpy
     6570    lib/umqtt.py
      614    lib/utils.mpy
     1691    lib/utils.py
             lib/websocket/
      641    lib/websockets.mpy
      976    lib/websockets.

In [41]:
from lib.decoding import CCA

### Using a Runner for experimentation and logging
The a `Runner` is encapsulates the core functions in this EEG system, including peripheral setup, sampling, signal processing, logging and memory management. The `OnlineRunner` offers mostly the same functionality as the standard `Runner` class, except it allows for logging and other communication with a remote server - either on the Internet or on your local network.

#### Offline functionality
The standard `Runner` is good for testing core functionality without the need for remote logging. See below for initialisation and execution.

In [42]:
from lib.runner import OnlineRunner, Runner

runner = Runner() # initialise a base runner
runner.setup() # setup peripherals and memory buffers

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498


In [19]:
%ls

Listing directory '/'.
     3116    .env
      139    boot.py
             lib/
      935    main.py


In [48]:
# start sampling and recording data (logging not setup in this case)
runner.run()

In [49]:
# see if runner has indeed started smapling
print(runner.is_sampling)

True


In [50]:
# display the contents of the output buffer - this will be updated internally by the runner
# at a rate determined by the sampling frequency and sample buffer size (typically every 1s)

print(runner.output_buffer)

[0.01330769, 0.531615, 1.472553, 1.322852, 5.377353, 6.63016, -0.962748, 6.276139, 4.775703, -7.763105, 3.212142, 2.703174, -5.692567, -1.22802, 4.109945, -1.899578, -4.800727, 8.060905, 3.860339, 1.055837, 6.500959, 0.7379609, -2.585894, -1.864251, -7.464678, -3.898554, -2.715072, -9.747927, 2.274868, -2.461463, 4.217269, 10.34527, -2.337288, 11.2823, 0.03975558, 0.04130304, 3.391908, -11.5179, 0.6564133, -7.058274, -4.963298, 4.648061, -5.870523, 4.198328, 3.4022, -0.9603409, 7.652535, -0.9741123, 1.917029, -0.343415, -11.08197, -2.590955, -9.467142, -1.69321, 1.881659, -7.342432, 7.894325, -0.06241745, 2.543673, 7.749337, -3.054949, 7.129214, -1.625226, -4.395927, 0.009568786, 0.8080921, 3.581329, 1.462777, -5.347327, -4.473403, -5.726441, 0.1042426, 10.27883, -0.7721684, 3.065126, 12.66041, 4.687962, 5.035817, 10.16542, 6.496332, -6.136485, -4.557409, -3.376487, -10.86264, -4.376719, -4.713753, -3.540468, 4.933814, -0.1367997, 6.079503, 7.40438, -3.153892, 6.679387, -1.839524, -4.3

In [51]:
# decode the contents of the output buffer. This will return a dictionary whose keys are the
# candidate SSVEP stimulus frequencies and the values, their corresponding output correlation
# estimate

print(runner.decode())

{12: 0.1099, 10: 0.10414, 7: 0.10239}


In [54]:
# stop runner
runner.stop()

#### Simple decoding loop
In order to test online decoding, here is a basic synchronous loop-based option. Interrupt the cell to stop the infinite loop.

In [98]:
import utime as time
from lib.runner import Runner

decode_period_s = 2 # decode every x seconds

runner = Runner() # initialise a base runner
runner.setup()

runner.run() # start sampling

try:
    while True:
        time.sleep(decode_period_s)
        decoded = runner.decode()
        print(decoded,": ", max(decoded,key=decoded.get))
except KeyboardInterrupt:
    print('received SIGINT - stopping')

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
{12: 0.05586, 10: 0.07374, 7: 0.0478} :  10
{12: 0.1193, 10: 0.10566, 7: 0.07898999} :  12
.{12: 0.16063, 10: 0.094, 7: 0.11783} :  12
{12: 0.09403, 10: 0.07389, 7: 0.08966} :  12
.{12: 0.13845, 10: 0.13321, 7: 0.05538} :  12
{12: 0.1462, 10: 0.12491, 7: 0.0983} :  12
.{12: 0.09631, 10: 0.07609, 7: 0.12846} :  7
{12: 0.10075, 10: 0.08919, 7: 0.11311} :  7
{12: 0.03189, 10: 0.03725, 7: 0.05136} :  7
.{12: 0.04783, 10: 0.05685, 7: 0.0753} :  7
{12: 0.02614, 10: 0.06317, 7: 0.03992} :  10
.{12: 0.00714, 10: 0.0303, 7: 0.02334} :  10


*** Sending Ctrl-C

received SIGINT - stopping


In [99]:
runner.stop()

In [ ]:
NORMAL CCA
7hz
{12: 0.09985, 10: 0.08019, 7: 0.05285} :  12
{12: 0.30064, 10: 0.13745, 7: 0.09656} :  12
.{12: 0.26374, 10: 0.1088, 7: 0.11329} :  12
{12: 0.28618, 10: 0.11051, 7: 0.14379} :  12
.{12: 0.21606, 10: 0.10466, 7: 0.11286} :  12
{12: 0.27251, 10: 0.09596, 7: 0.10037} :  12
.{12: 0.10679, 10: 0.11819, 7: 0.1399} :  7
{12: 0.22721, 10: 0.17712, 7: 0.09253} :  12
{12: 0.12842, 10: 0.1086, 7: 0.06267} :  12
    
10hz
12: 0.09736, 10: 0.04251, 7: 0.05008} :  12
{12: 0.02714, 10: 0.03751, 7: 0.01752} :  10
.{12: 0.09417, 10: 0.09463, 7: 0.12726} :  7
{12: 0.20466, 10: 0.15183, 7: 0.10772} :  12
.{12: 0.10881, 10: 0.08183001, 7: 0.08652} :  12
{12: 0.28923, 10: 0.07327, 7: 0.0991} :  12
.{12: 0.10173, 10: 0.05764, 7: 0.09438} :  12
{12: 0.16551, 10: 0.14892, 7: 0.08412999} :  12
{12: 0.08233, 10: 0.04017, 7: 0.10037} :  7
.{12: 0.09879, 10: 0.04898, 7: 0.10917} :  7
{12: 0.05018, 10: 0.09307, 7: 0.17937} :  7

12hz
{12: 0.05586, 10: 0.07374, 7: 0.0478} :  10
{12: 0.1193, 10: 0.10566, 7: 0.07898999} :  12
.{12: 0.16063, 10: 0.094, 7: 0.11783} :  12
{12: 0.09403, 10: 0.07389, 7: 0.08966} :  12
.{12: 0.13845, 10: 0.13321, 7: 0.05538} :  12
{12: 0.1462, 10: 0.12491, 7: 0.0983} :  12
.{12: 0.09631, 10: 0.07609, 7: 0.12846} :  7
{12: 0.10075, 10: 0.08919, 7: 0.11311} :  7
{12: 0.03189, 10: 0.03725, 7: 0.05136} :  7
.{12: 0.04783, 10: 0.05685, 7: 0.0753} :  7
{12: 0.02614, 10: 0.06317, 7: 0.03992} :  10
.{12: 0.00714, 10: 0.0303, 7: 0.02334} :  10

### Testing your WiFi connection
In order to connect to a local WiFi network, you'll need to supply your network SSID and password in a `.env` file on the board. Doing this is easy: 
1. On your computer, create a `.env` file using `touch .env`. Update the `.env` file with the required fields:
    
    ```bash
    #.env 
    WIFI_SSID=<your network name>
    WIFI_PASSWORD=<your network password>
    
    ```
    
2. Send this file to your target device using the following command:
    ```ipython
%sendtofile --source lib/.env lib/.env  --binary
```

You may need to update the local (source) path to your `.env` file depending on where you created/stored it.

In [110]:
%sendtofile --source /Users/rishil/Desktop/.env lib/.env  --binary

Sent 55 bytes in 2 chunks to lib/.env.


In [115]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

network config: ('192.168.0.68', '255.255.255.0', '192.168.0.1', '194.168.4.100')


#### Online Runner
Now that you've established network connectivitiy, you can test out an `OnlineRunner`. In order to test web logging to a remote server, we can use a basic HTTP logger. However, this obviously needs an API/server willing to accept our requests. There is a basic logging API using `Flask` in `/eeg_lib/logging_server.py`. You can run it using `python logging_server.py` which will spin up a development server on the predefined port (5000 or 5001). Then, just configure your `OnlineRunner` with the appropriate logger params and you're set.

In [116]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

api_host = "http://192.168.0.13:5001/" # make sure the port corresponds to your logging server configuration
log_params = dict(server=api_host, log_period=4, logger_type=logger_types.HTTP, send_raw=True, session_id='eeg_log')

runner = OnlineRunner()
runner.setup(**log_params)

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
network config: ('192.168.0.68', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [119]:
# start the runner - you should see requests being made to your local server
runner.run()

In [120]:
runner.stop()

## Experimentation

In [16]:
import urandom

def synth_x(f, Ns, noise_power=0.5, fs=250):
    """
    generate a synthetic signal vector
    
    args:
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    """
    t = np.arange(0, Ns/fs, 1/fs)
    return np.sin(t*2*np.pi*f)*(1+urandom.random()*noise_power)

def synth_X(f, Nc, Ns, noise_power=0.5, fs=200, f_std=0.02):
    """
    Generate a matrix of several variations of the same target signal. This is used
    to simulate the measurement of a common signal over multiple EEG channels 
    that have different SNR characteristics.
    
    args:
    f [float]: target frequency of synthetic signal (Hz)
    Nc [int]: number of channels
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    fs [float]: sampling frequency (Hz)
    f_std [float]: standard dev. of freq. in generated signal across channels to simulate interference from other frequency components over different channels
    """
    X = []
    for i in range(Nc): # simulate noisy sinusoids with varying SNR across Nc channels
        f_i = f*(1+urandom.random()*f_std)
        x = synth_x(f_i, Ns, noise_power=noise_power)

        X.append(x)
        
    return np.array(X)

In [23]:
from lib.decoding import harmonic_reference, CCA
from lib.computation import solve_gen_eig_prob
from ulab import numpy as np

gc.collect()

f0 = 7
stim_freqs = [7, 10, 12]
fs = 200

# generate a synthetic noisy signal at 7Hz over 2 channels with 100 samples
X_test = synth_X(7, 2, 100)

cca = CCA(stim_freqs, fs)

print({f: vals[0] for f, vals in cca.compute_corr(X_test).items()})


{12: 0.09335411, 10: 0.1326894, 7: 0.5633128}


In [18]:
from lib.decoding import harmonic_reference

X = X_test
Y = harmonic_reference(7, 200, np.max(X_test.shape), Nh=2, standardise_out=True)

In [19]:
from lib.decoding import harmonic_reference
from lib.computation import max_eig

X = X_test
Y = harmonic_reference(7, 200, np.max(X_test.shape), Nh=2, standardise_out=True)

Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
M2 = np.dot(np.linalg.inv(Cyy), Cyx)

lam, _ = max_eig(np.dot(M1, M2), 20)

In [20]:
print(lam)

0.3366819


In [21]:
def zeros_like(A):
    return np.zeros(A.shape)

def block_diag(X, Y, reverse=False):
    if not reverse:
        X = np.concatenate((X, zeros_like(X)), axis=1)
        Y = np.concatenate((zeros_like(Y), Y), axis=1)
    else:
        X = np.concatenate((zeros_like(X), X), axis=1)
        Y = np.concatenate((Y, zeros_like(Y)), axis=1)
    return np.concatenate((X, Y), axis=0)

In [52]:
%ls --recursive lib

Listing directory 'lib'.
       53    lib/.env
        0    lib/__init__.py
             lib/__pycache__/
     4006    lib/computation.py
      446    lib/config.py
     2323    lib/decoding.py
      855    lib/diagnostics.py
     3503    lib/logging.py
     1254    lib/networking.py
     5697    lib/peripherals.py
    27364    lib/requests.py
     7664    lib/runner.py
     1911    lib/scheduling.py
     2067    lib/signal.py
     6570    lib/umqtt.py
     1691    lib/utils.py
             lib/websocket/
      976    lib/websockets.py


In [22]:
%sendtofile --source lib/.env .env  --binary

[leftinbuffer] ['>']
[leftinbuffer] ['MicroPython v1.16-141-g224ac355c-dirty on 2021-07-28; ESP32 module with ESP32']
[leftinbuffer] ['Type "help()" for more information.']
[leftinbuffer] ['>>> ']


In [102]:
%sendtofile lib/runner.py --source lib/runner.py

Sent 233 lines (7664 bytes) to lib/runner.py.


In [82]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython 97a7cc2-dirty on 2021-11-26; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython 97a7cc2-dirty on 2021-11-26; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [56]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute]
                   [--source [SOURCE]] [--quiet] [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US

In [58]:
json.loads("/Users/rishil/Desktop/EEG-decoding/eeg_lib/logs/log_data.json")

Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
OSError: stream operation not supported


In [115]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
# %serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200


Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 could not open port /dev/tty.usbserial-0001: [Errno 16] Resource busy: '/dev/tty.usbserial-0001'

Try one of these ports as --port= 
  /dev/cu.usbserial-0001
  /dev/cu.BLTH
  /dev/cu.Bluetooth-Incoming-Port

In [136]:
from machine import Pin

[leftinbuffer] ['b"G\\x01\\x02\\x91\\xa5\\xd1\\x85\\xd1\\xa5\\xbd\\xb9\\x01\\x00\\x0e\\xbd\\xc9\\x95\\x81 0 panic\'ed (InstrFetchProhibited). Exception was unhandled.\\r\\nCore 0 register dump:\\r\\nPC      : 0x00000ets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nload:0x3fff0018,len:4\\r\\nload:0x3fff001c,len:4252\\r\\nload:0x40078000,len:11920\\r\\nload:0x40080400,len:3344\\r\\nentry 0x4008060c\\r\\nets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RE\\x03\\xfbx13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\x00ets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode

In [137]:
p26 = Pin(26, Pin.OUT)
p13 = Pin(13, Pin.OUT) 

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'Pin' isn't defined


In [98]:
p26.on()
p13.on()

In [99]:
p26.off()
p13.off()

In [75]:
import time

In [81]:
counter = 0
while 1==1:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 9, in <module>
KeyboardInterrupt: 


In [112]:
print(1)

No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [143]:
%sendtofile --source /Users/rishil/Desktop/pythonfiles/boot1.py boot.py  --binary

Sent 0 bytes in 1 chunks to boot.py.
